In [2]:
from tps import common
from tps import tradebook as tb
import pandas as pd
import re

In [6]:
g_symbol_df = common.load_symbol_csv('./symbol_ib.txt')
ohlc_dct = tb.get_symbol_ohlc(g_symbol_df)

*** load ohlc from ./cache/AAPL_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/AMZN_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/MSFT_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/GOOGL_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/FB_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/ADBE_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/CRM_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/NFLX_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/TSLA_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/BABA_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/NTES_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/BIDU_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/SE_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/NVDA_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/AVGO_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/INTC_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/MU_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/SMH_STK_ohlc_IB1D.csv ***
*** load ohlc from ./cache/SPY_STK_

In [3]:
cfg="""{
"algo":[
    {
      "name":"macd",
      "param":{}
    },
    {
      "name":"nhnl",
      "param":{
          "pct":"0.99"
       }
    }
  ]
}"""
import json
rule=json.loads(cfg)

In [ ]:
algo_dct=tb.load_algo_pipeline(rule)

In [ ]:
ohlc_dct

In [ ]:
import pandas as pd
df_raw = {'symbol':[],'px':[]}

for symbol,ohlc in ohlc_dct.items():
    if ohlc.empty:
        print(symbol, "ohlc is empty")
        continue

    for an,algo_inx in algo_dct.items():
        ind_dct = algo_inx.run_algo(ohlc)
        #ind_dct['px'] = ohlc['Close'].iloc[-1]
        if ind_dct:
            for cn in ind_dct:
                if cn not in df_raw:
                    df_raw[cn]=[]
                lst = df_raw[cn]
                lst.append(ind_dct[cn])

    df_raw['symbol'].append(symbol)
    df_raw['px'].append(ohlc['Close'].iloc[-1])
df= pd.DataFrame.from_dict(df_raw, orient='columns', dtype=None)

In [ ]:
df.sort_values(by=['ma10_sig'], inplace=True, ascending=['True'])
df

In [ ]:
cfg = """{  
  "criteria_up":[
    "ma10_sig>0",
    "slo10>slo20",
    "slo20>slo50"
  ],
  "criteria_nhnl":[
    "ma10_sig>0",
    "sth_sig<8"
  ],
  "criteria_ma":[
    "ma10_sig>0",
    "ma20_sig>0",
    "ma50_sig>0"
  ],
  "criteria_sth":[
    "px>0.98*sth"
  ],
  "criteria_200":[
    "px>0.98*ma200",
    "px<1.02*ma200"
  ],
  "criteria":[
    "ma10_sig>0",    
    "lth_sig>0"
  ],
  "columns":[
    "symbol","px",
    "macd","ma10%","ma20%","ma50%","ma10","ma20","ma50","ma200","volra20","vol20_sig","ma10_sig","ma20_sig","ma50_sig","slo10","slo20","slo50",
    "sth","lth","stl","ltl","lth_sig","sth_sig","ltl_sig","stl_sig"
  ],
  "sort":[
    {"ma10_sig":"True"},
    {"ma50_sig":"True"}
  ]
}"""
import json
rule=json.loads(cfg)

In [ ]:
import re
if 'criteria' in rule:
    criteria = rule['criteria']
else:
    criteria = []

collst = []
if not criteria:
    print("criteria is empty,...take the original table")
else:
    # filter by dynamic criteria string
    crstr = ""
    pattern1 = "([a-zA-Z][A-Za-z0-9-_]*)"
    pattern2 = "[></]"

    for cr in criteria:
        print("processing cr", cr)
        if cr[0] == '@':  # TODO handle parameter
            continue
        collst = re.findall(pattern1, cr) # find all related columns
        ration = re.findall(pattern2, cr)
        if len(ration) != 0:
            cr0 = re.sub(pattern1, r"df['\1']", cr)  # put df[] surround pattern / substitute
            if crstr == "":  # first criteria
                crstr = crstr + "(" + cr0 + ") "
            else:
                crstr = crstr + "& (" + cr0 + ") "
    print("\tto evaluate criteria(logical) = %s" % crstr)
    if crstr != "":
        df = df[eval(crstr)]
    # ===========================================================
#print(g_rule)
if 'columns' in rule:
    output_cols = rule['columns']
    if not output_cols:
        output_cols.extend(df.columns.values)

    #print('===========================================',output_set)
    for col in collst:
        if col not in output_cols:  # keep origin order
            output_cols.append(col)
    df = df[output_cols]
if 'sort' in rule:
    sort_list = rule['sort']
    if sort_list:
        sort_cols=[]
        asc_list=[]
        for dct in sort_list:
            for key,value in dct.items():
                sort_cols.append(key)
                asc_list.append(False if value=="False" else True)
        df.sort_values(by=sort_cols, inplace=True, ascending=asc_list)
        pass
dfscan = df

In [ ]:
dfscan

In [ ]:
import plotly.graph_objects as go
import pandas as pd

ohlc=ohlc_dct['AAPL']
fig = go.Figure(data=go.Ohlc(x=ohlc['Date'],
                    open=ohlc['Open'],
                    high=ohlc['High'],
                    low=ohlc['Low'],
                    close=ohlc['Close']))
fig.show()

# SCTR
SCTR是用来表达一个股票在一组股票里的相对强度，sctr_rank 为100表明最强的那支股票
https://school.stockcharts.com/doku.php?id=technical_indicators:sctr

In [7]:
import json
cfg="""{
"algo":[
    {
      "name":"sctr",
      "param":{}
    }
  ]
}"""

rule=json.loads(cfg)
algo_dct=tb.load_algo_pipeline(rule)

In [8]:
df=tb.run_algo(ohlc_dct,algo_dct)

processing AAPL
processing AMZN
processing MSFT
processing GOOGL
processing FB
processing ADBE
processing CRM
processing NFLX
processing TSLA
processing BABA
processing NTES
processing BIDU
processing SE
processing NVDA
processing AVGO
processing INTC
processing MU
processing SMH
processing SPY
processing IWM
processing QQQ
processing JPM
processing BLK
processing WFC
processing MA
processing MS
processing V
processing GS
processing JNJ
processing MRK
processing PG
processing HD
processing MMM
processing WMT
processing BA
processing DIS


In [9]:
df

,symbol,px,stok,sctr,sctr_rank,sctr_r3,sctr_r5,sctr_r10,sctr_r15,sctr_r20,sctr_r25,sctr_r30
0,TSLA,804.00,65.856,136.180,100.00,100.00,100.00,100.00,100.00,100.00,100.00,100.00
1,NVDA,270.78,89.585,42.028,97.22,97.22,94.44,91.67,91.67,94.44,94.44,94.44
2,AAPL,324.87,77.690,32.801,94.44,94.44,97.22,97.22,97.22,97.22,97.22,97.22
3,SE,47.95,78.002,32.440,91.67,91.67,91.67,94.44,94.44,80.56,83.33,77.78
4,INTC,67.44,79.494,27.502,88.89,88.89,86.11,86.11,88.89,41.67,52.78,75.00
5,MSFT,183.71,37.946,26.254,86.11,86.11,88.89,83.33,66.67,75.00,69.44,69.44
6,MU,59.33,55.645,24.797,83.33,66.67,80.56,55.56,83.33,86.11,91.67,91.67
7,NTES,354.33,68.333,23.426,80.56,77.78,83.33,88.89,86.11,91.67,88.89,83.33
8,ADBE,374.84,78.172,23.345,77.78,75.00,75.00,77.78,69.44,58.33,66.67,61.11
9,BLK,568.55,84.997,23.312,75.00,83.33,77.78,66.67,58.33,47.22,38.89,36.11


In [ ]:
# 过滤
sctr>sctr5天前>sctr10天前，表示sctr排名在上升


In [ ]:
cfg = """{  
  "criteria":[
    "sctr_rank>sctr_r5",
    "sctr_r5>sctr_r10"
  ],
  "columns":[
    "symbol","px",
    "stok","sctr","sctr_rank","sctr_r5","sctr_r10","sctr_r30"
  ],
  "sort":[
    {"sctr_rank":"False"}
  ]
}"""
import json
criteria=json.loads(cfg)

In [13]:
df_scan=tb.run_scan(df,criteria)

processing cr sctr_rank>sctr_r5
processing cr sctr_r5>sctr_r10
	to evaluate criteria(logical) = (df['sctr_rank']>df['sctr_r5']) & (df['sctr_r5']>df['sctr_r10']) 


In [14]:
df_scan


,symbol,px,stok,sctr,sctr_rank,sctr_r5,sctr_r10,sctr_r30
1,NVDA,270.78,89.585,42.028,97.22,94.44,91.67,94.44
6,MU,59.33,55.645,24.797,83.33,80.56,55.56,91.67
11,SMH,151.34,92.998,22.556,69.44,58.33,50.00,88.89
12,NFLX,381.40,83.767,20.369,66.67,50.00,38.89,5.56
18,BIDU,135.18,72.101,17.580,50.00,44.44,19.44,72.22
28,IWM,168.55,84.949,10.121,22.22,19.44,16.67,25.00


In [ ]:
ohlc

In [ ]:
from algo import tps_ind

In [ ]:
ohlc = tps_ind.MA(ohlc, [50, 200])

In [ ]:
ohlc

In [ ]:
ohlc['Close'].values